In [ ]:
import google.generativeai as genai

# genai.configure(api_key="YOUR_API_KEY") # Configure your Gemini API key

def get_gemini_embedding(text, task_type="RETRIEVAL_DOCUMENT"):
    """Generates an embedding for the given text using Gemini's embedding model."""
    try:
        response = genai.embed_content(
            model="models/embedding-001",
            content=text,
            task_type=task_type
        )
        return response['embedding']
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None

mind_map_nodes_embeddings = {}
# Assume 'mind_map_data' is loaded from your JSON mind map file
for node_id, node_data in mind_map_data['nodes'].items():
    node_text = f"{node_data['title']}. {node_data['description']}"

    # Add parent context
    if 'parent_id' in node_data and node_data['parent_id'] in mind_map_data['nodes']:
        node_text += f" Parent: {mind_map_data['nodes'][node_data['parent_id']]['title']}."

    # Add children context
    if 'child_ids' in node_data:
        children_titles = [mind_map_data['nodes'][cid]['title'] for cid in mind_map_data['nodes'] if cid in node_data['child_ids']]
        if children_titles:
            node_text += f" Children: {', '.join(children_titles)}."

    embedding = get_gemini_embedding(node_text, task_type="RETRIEVAL_DOCUMENT")
    if embedding:
        mind_map_nodes_embeddings[node_id] = {
            "text": node_text, # Store for debugging/inspection
            "embedding": embedding,
            "title": node_data['title'],
            "description": node_data['description'],
            "related_document_ids": node_data.get('related_document_ids', [])
        }

## Gemini Embeddings for Document Chunks

Similarly, each preprocessed document chunk is converted into a vector embedding using the same models/embedding-001 with task_type="RETRIEVAL_DOCUMENT". This ensures semantic compatibility between node embeddings and document chunk embeddings.

In [ ]:
document_chunks_embeddings = []
# Assume 'processed_documents' is a dictionary {doc_id: [chunk_text1, chunk_text2, ...]}
for doc_id, chunks in processed_documents.items():
    for i, chunk_text in enumerate(chunks):
        embedding = get_gemini_embedding(chunk_text, task_type="RETRIEVAL_DOCUMENT")
        if embedding:
            document_chunks_embeddings.append({
                "doc_id": doc_id,
                "chunk_index": i,
                "text": chunk_text,
                "embedding": embedding
            })


# Vector Database Indexing
All generated embeddings (from mind map nodes and document chunks) are indexed in a vector database. For this work, we utilize Vertex AI Vector Search (formerly Matching Engine), which provides highly scalable and efficient nearest-neighbor search capabilities, seamlessly integrated within the Google Cloud ecosystem. Each indexed vector is associated with metadata, including:

For mind map nodes: node_id, title, description, related_document_ids.

For document chunks: doc_id, chunk_index, original_text.
This metadata is crucial for the retrieval phase.

# **3.4. Hybrid Retrieval Strategy**
The core novelty of MM-RAG lies in its hybrid retrieval strategy, which leverages the mind map's structure to guide and refine the document retrieval process. This two-stage approach ensures that the context provided to the LLM is not only semantically relevant but also structurally grounded within the organizational knowledge framework.

A. Stage 1: Mind Map Node Retrieval:
When a user poses a query, its embedding is first generated using models/embedding-001 with task_type="RETRIEVAL_QUERY". This query embedding is then used to perform a similarity search against the indexed mind map node embeddings in Vertex AI Vector Search. The top-K most similar mind map nodes are retrieved. This step effectively identifies the most relevant conceptual areas within the offboarding knowledge base.

In [ ]:
def retrieve_mind_map_nodes(query_embedding, vector_db_client, k=5):
    """Retrieves top-K mind map nodes similar to the query."""
    # This is a conceptual call; actual implementation depends on Vertex AI Vector Search API
    results = vector_db_client.query(query_embedding, index_name="mind_map_nodes_index", top_k=k)
    return results # Returns list of {node_id, score, metadata}

B. Stage 2: Contextualized Document Chunk Retrieval:

In [ ]:
def retrieve_document_chunks(query_embedding, vector_db_client, relevant_doc_ids, m=10):
    """Retrieves top-M document chunks from a filtered set of documents."""
    # Conceptual call; actual implementation would filter by doc_id metadata
    # Some vector DBs allow direct filtering during query, others require post-filtering
    results = vector_db_client.query(
        query_embedding,
        index_name="document_chunks_index",
        top_k=m,
        filter_by_metadata={'doc_id': relevant_doc_ids}
    )
    return results # Returns list of {doc_id, chunk_index, text, score}

def get_augmented_context(user_query, mind_map_nodes_data, doc_chunks_data):
    query_embedding = get_gemini_embedding(user_query, task_type="RETRIEVAL_QUERY")

    # Stage 1: Retrieve mind map nodes
    retrieved_nodes_info = retrieve_mind_map_nodes(query_embedding, vector_db_client, k=5)

    relevant_doc_ids = set()
    mind_map_context_str = "Relevant Mind Map Concepts:\n"
    for node_info in retrieved_nodes_info:
        node_id = node_info['node_id']
        node_metadata = mind_map_nodes_embeddings[node_id] # Assuming we store metadata in a dict
        mind_map_context_str += f"- {node_metadata['title']}: {node_metadata['description']}\n"
        relevant_doc_ids.update(node_metadata['related_document_ids'])

    # Stage 2: Retrieve document chunks based on relevant documents
    retrieved_chunks_info = retrieve_document_chunks(query_embedding, vector_db_client, list(relevant_doc_ids), m=10)

    document_context_str = "Relevant Document Snippets:\n"
    for chunk_info in retrieved_chunks_info:
        document_context_str += f"- {chunk_info['text']}\n" # Assuming 'text' is part of retrieved info

    return mind_map_context_str + "\n" + document_context_str

# Augmented Generation with Gemini Pro


The final stage involves leveraging Gemini Pro to synthesize a comprehensive and accurate response. The retrieved context, comprising both the mind map node descriptions and the relevant document chunks, is concatenated and passed to Gemini Pro along with the user's original query.

The prompt engineering for Gemini Pro is crucial. We instruct the model to:

Answer the question based only on the provided context.

Clearly state if the information is insufficient to answer the question.

Structure the answer clearly (e.g., bullet points, short paragraphs).

Prioritize factual accuracy and avoid speculation.


In [ ]:
import google.generativeai as genai

# genai.configure(api_key="YOUR_API_KEY") # Ensure API key is configured

def generate_response(user_query, augmented_context):
    """Generates a response using Gemini Pro based on the query and augmented context."""
    prompt = f"""
    You are an intelligent knowledge transfer assistant.
    Based *only* on the following context, answer the user's question.
    If the context does not contain enough information to answer the question, state that clearly.

    ---
    Context:
    {augmented_context}
    ---

    User Question: {user_query}

    Answer:
    """

    model = genai.GenerativeModel('gemini-pro')
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating content with Gemini Pro: {e}")
        return "An error occurred while generating the response."